In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Wine Reviews Data Exploration
## Data
This is the dataset I found on [Kaggle](https://www.kaggle.com/zynicide/wine-reviews), the original dataset was scraped from [WineEnthusiast](https://www.winemag.com/?s=&drink_type=wine). The collection of dataset was inspired by the movie [Somm](https://www.imdb.com/title/tt2204371/), which is a documentary movie that describes four sommeliers attempt to pass the Master Sommelier exam, a test with one of the lowest pass rates in the world.

The reason I chose this dataset for my exploration report is that I found wine tasting a pretty important social skills and the more knowledge you have regarding wines, people seems to show more interests and repects of you. This dataset consists of around 280K observations and I think this is a good start of my first capstone project for exploratory data analysis practice.

The dataset conssits of 2 tables, which were basically scraped in different times, both tables contain country, description, designation, points, price, province, regions, variety and winery. One of the table has more information about taster name, taster twitter handle and title of the wine review. 



## Analytic Questions
### 1. What are the type of grapes that are mostly used to make the wine in each country? Further break down the information for US by state

